# Welcome to Polyglot .net interactive demo 

* Author: Julie Koesmarno [@MsSQLGirl](http://twitter.com/mssqlgirl) / [MsSQLGirl.com](http://MsSQLGirl.com), Alan Yu [@AlanYuSQL](http://twitter.com/AlanYuSQL)
* Date last updated: Nov 9, 2021

This is a demo notebook for "What's the latest in .net interactive" at Dot Net Conference 2021.

Let's get the party started 🎉👩🏻‍💻👨🏻‍💻💻
* Hello Data!
* Let's talk..
* We're back, the Sequel!

# Setup

* VSCode v1.62.1
* VSCode extensions:
    * .net Interactive extension v1.0.2559020
    * Install Data Table extension v1.8.0
    * Install Jupyter extension in VSCode v2021.10.1001414422
* Nuget package: 
    * KQL 1.0.0-beta.21558.1
    * SQL 1.0.0-beta.21559.2 

<hr/>

# KQL Demo

Load the latest KQL Extension for .NET interactive (Version 1.0.0-beta.21558.1)

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.Kql, 1.0.0-beta.21558.1" 

Installed Packages Microsoft.DotNet.Interactive.Kql, 1.0.0-beta.21558.1

Loading extensions from `Microsoft.DotNet.Interactive.Kql.dll`

Query Microsoft Kusto Server databases. 
 This extension adds support for connecting to Microsoft Kusto Server databases using the #!connect kql magic command. For more information, run a cell using the #!kql magic command.

## Connect to ADX cluster

Connecting to Samples database. 

In [ ]:
#!connect kql --kernel-name KustoHelp --cluster "https://help.kusto.windows.net" --database "Samples"

Kernel added: #!kql-KustoHelp

## Analyze StormEvents data

In [ ]:
#!kql-KustoHelp --name weatherData
StormEvents 
| summarize count() by bin(StartTime, 1d), State
| limit 10


## Share KQL data to C# 

weatherData shows as TabularDataResource type in C#

In [ ]:
#!share --from kql-KustoHelp weatherData
weatherData[0].Display();
// weatherData[0].GetType().Display();

In [ ]:
#!kql-KustoHelp
StormEvents 
| summarize EventCount = count() by EventType, State, MonthNumber = getmonth(StartTime)
| where EventType in ("Hail", "Flash Flood", "Drought", "Winter Weather", "Winter Storm")
| where State in ("WASHINGTON", "CALIFORNIA", "TEXAS")



# Let's try conneting to SQL Server!

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer, *-*"

Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.21559.2

Loading extensions from `Microsoft.DotNet.Interactive.SqlServer.dll`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

In [ ]:
#!connect mssql -h

## Connect to a SQL Server instance

In [ ]:
#!connect mssql --kernel-name retailData "Server=SQLTOOLS2019-3;Database=keep_WideWorldImporters;Trusted_Connection=Yes;"

In [ ]:
#!sql-retailData

SELECT TOP 10
     i.InvoiceDate
    ,i.InvoiceID
    ,si.StockItemName
    ,gr.StockGroupName
    ,il.ExtendedPrice AS Revenue
    ,il.LineProfit AS Profit
    ,il.Quantity
    ,il.LineProfit * 100.0 / il.ExtendedPrice AS [Margin%]    
FROM Sales.InvoiceLines il
    INNER JOIN Sales.Invoices i
        ON i.InvoiceID = il.InvoiceID
    INNER JOIN Warehouse.StockItems_Archive si
        ON si.StockItemID = il.StockItemID 
        AND i.InvoiceDate BETWEEN si.ValidFrom AND si.ValidTo    
    INNER JOIN Warehouse.StockItemStockGroups sg
        ON sg.StockItemID = si.StockItemID        
    INNER JOIN Warehouse.StockGroups gr
        ON gr.StockGroupID = sg.StockGroupID
        AND i.InvoiceDate BETWEEN gr.ValidFrom AND gr.ValidTo
WHERE i.InvoiceDate >= '20160101'
AND  i.InvoiceDate < '20160201'
ORDER BY i.InvoiceID, i.InvoiceDate, si.StockItemName

## Interactive experience in .net interactive

Get input from users (in C#)

In [ ]:
var inputStockGroup = await GetInputAsync();
System.Console.WriteLine(inputStockGroup);


Pass the input parameter from C# to SQL!

In [ ]:
#!sql-retailData
#!share --from csharp inputStockGroup

SELECT 
     gr.StockGroupName
    ,SUM(il.Quantity) AS TotalQuantity
    ,SUM(il.LineProfit) * 100.0 / SUM(il.ExtendedPrice) AS [Margin%]    
FROM Sales.InvoiceLines il
    INNER JOIN Sales.Invoices i
        ON i.InvoiceID = il.InvoiceID
    INNER JOIN Warehouse.StockItems_Archive si
        ON si.StockItemID = il.StockItemID 
        AND i.InvoiceDate BETWEEN si.ValidFrom AND si.ValidTo    
    INNER JOIN Warehouse.StockItemStockGroups sg
        ON sg.StockItemID = si.StockItemID        
    INNER JOIN Warehouse.StockGroups gr
        ON gr.StockGroupID = sg.StockGroupID
        AND i.InvoiceDate BETWEEN gr.ValidFrom AND gr.ValidTo
WHERE gr.StockGroupName = @inputStockGroup
GROUP BY gr.StockGroupName
